In [112]:
module Ev3dev

import Base: read, parse, write

parse(::Type{ASCIIString}, s::ASCIIString) = s
parse(::Type{Vector{ASCIIString}}, s::ASCIIString) = split(s)

immutable Brick
    root_path::AbstractString
end

@inline novalidate(x) = true

immutable Attribute{T}
    name::Symbol
    read::Bool
    write::Bool
    validator::Function
end

Attribute{T}(name::Symbol, ::Type{T}, read::Bool=true, write::Bool=false, validator::Function=novalidate) = Attribute{T}(name, read, write, validator)

immutable WrappedStream{T}
    stream::IOStream
    validator::Function
end

function WrappedStream{T}(port_path::AbstractString, attribute::Attribute{T})
    path = joinpath(port_path, string(attribute.name))
    isfile(path) || error("file not found: $path")
    attribute.read && (isreadable(path) || error("read access for attribute $(attribute.name) was requested, but file is not readable"))
    attribute.write && (iswritable(path) || error("write access for attribute $(attribute.name) was requested, but file is not writable"))

    stream = open(path, attribute.read, attribute.write, false, attribute.write, false)
    WrappedStream{T}(stream, attribute.validator)
end

function read{T}(attr::WrappedStream{T})
    seekstart(attr.stream)
    parse(T, readchomp(attr.stream))
end

function write{T}(attr::WrappedStream{T}, val::T)
    @assert attr.validator(val)
    seekstart(attr.stream)
    write(attr.stream, string(val))
    flush(attr.stream)
end

write{T}(attr::WrappedStream{T}, val) = write(attr, convert(T, val))

abstract Device

immutable Port
    brick::Brick
    path::AbstractString
    streams::Dict{Symbol, WrappedStream}
end

function Port(brick::Brick, path::AbstractString, 
    attributes::AbstractArray{Attribute})
    
    streams = Dict{Symbol, WrappedStream}()
    for attribute in attributes
        streams[attribute.name] = WrappedStream(joinpath(brick.root_path, path), attribute)
    end
    Port(brick, path, streams)
end

read(port::Port, name::Symbol) = read(port.streams[name])
write(port::Port, name::Symbol, value) = write(port.streams[name], value)
read(device::Device, name::Symbol) = read(device.port, name)
write(device::Device, name::Symbol, value) = write(device.port, name, value)

function find_device{T}(::Type{T}, brick::Brick, address::AbstractString, path_to_search::AbstractString)
    path = joinpath(brick.root_path, path_to_search)
    for dir in readdir(path)
        device_address = open(joinpath(path, dir, "address")) do f
            readchomp(f)
        end
        if device_address == address
            return T(brick, dir)
        end
    end
    nothing
end

macro make_device(name, path, attributes)
    quote type $(esc(name)) <: Device
            port::Port
        end
        
        function $(esc(name))(brick::Brick, identifier::AbstractString) 
            $(esc(name))(Port(brick, joinpath($(path), identifier), Attribute[Attribute(x...) for x in $(attributes)]))
        end
        
        function $(esc(:find_device))(::Type{$(name)}, brick::Brick, address::AbstractString)
            find_device($(name), brick, address, $(path))
        end
        
        function make_accessors()
            for args in $(attributes)
                attribute = Attribute(args...)
                if attribute.read
                    expr = Expr(:(=), Expr(:call, attribute.name, Expr(:(::), :dev, $(esc(name)))), 
                    Expr(:call, :read, :dev, QuoteNode(attribute.name)))
                    eval(expr)
                end
                if attribute.write
                    expr = Expr(:(=), Expr(:call, symbol(attribute.name, :!), Expr(:(::), :dev, $(esc(name))), :value), 
                    Expr(:call, :write, :dev, QuoteNode(attribute.name), :value))
                    eval(expr)
                end
            end
        end
        make_accessors()
    end
end

@make_device Motor "sys/class/tacho-motor" [
    (:address, ASCIIString),
    (:commands, Vector{ASCIIString}),
    (:driver_name, ASCIIString),
    (:position, Int),
    (:count_per_rot, Int),
    (:duty_cycle, Int),
    (:duty_cycle_sp, Int, true, true, x -> x > 0),
    (:speed_sp, Int, true, true),
    (:position_sp, Int, true, true),
    (:polarity, ASCIIString, true, true, x -> x in Set(("normal", "inversed"))),
    (:command, ASCIIString, false, true),
    (:stop_action, ASCIIString, true, true)
    ]

@make_device Sensor "sys/class/lego-sensor" [
    (:address, ASCIIString),
    (:commands, Vector{ASCIIString}),
    (:driver_name, ASCIIString),
    (:decimals, Int),
    (:value0, Int),
    (:value1, Int),
    (:value2, Int),
    (:value3, Int),
    (:modes, Vector{ASCIIString}),
    (:bin_data, ASCIIString),
    (:bin_data_format, ASCIIString),
    (:poll_ms, Int, true, true, x -> x > 0)
    ]

end

import Ev3dev

In [113]:
brick = Ev3dev.Brick("/home/pi/ev3")

motor = Ev3dev.find_device(Ev3dev.Motor, brick, "outA")
ultrasound = Ev3dev.find_device(Ev3dev.Sensor, brick, "in1")

Ev3dev.Sensor(Ev3dev.Port(Ev3dev.Brick("/home/pi/ev3"),"sys/class/lego-sensor/sensor0",Dict{Symbol,Ev3dev.WrappedStream{T}}(:value3=>Ev3dev.WrappedStream{Int32}(IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value3>),Ev3dev.novalidate),:bin_data_format=>Ev3dev.WrappedStream{ASCIIString}(IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/bin_data_format>),Ev3dev.novalidate),:value0=>Ev3dev.WrappedStream{Int32}(IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value0>),Ev3dev.novalidate),:driver_name=>Ev3dev.WrappedStream{ASCIIString}(IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/driver_name>),Ev3dev.novalidate),:value2=>Ev3dev.WrappedStream{Int32}(IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value2>),Ev3dev.novalidate),:address=>Ev3dev.WrappedStream{ASCIIString}(IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/address>),Ev3dev.novalidate),:modes=>Ev3dev.WrappedStream{Array{ASCIIString,1}}(IOStream(<file /home/pi/ev3/sys/class/l

In [114]:
Ev3dev.value0(ultrasound)

569

In [115]:
read(motor, :commands)

7-element Array{SubString{ASCIIString},1}:
 "run-forever"   
 "run-to-abs-pos"
 "run-to-rel-pos"
 "run-timed"     
 "run-direct"    
 "stop"          
 "reset"         

In [116]:
Ev3dev.commands(motor)

7-element Array{SubString{ASCIIString},1}:
 "run-forever"   
 "run-to-abs-pos"
 "run-to-rel-pos"
 "run-timed"     
 "run-direct"    
 "stop"          
 "reset"         

In [117]:
write(motor, :speed_sp, 500)

IOStream(<file /home/pi/ev3/sys/class/tacho-motor/motor0/speed_sp>)

In [118]:
Ev3dev.command!(motor, "run-forever")

IOStream(<file /home/pi/ev3/sys/class/tacho-motor/motor0/command>)

In [119]:
Ev3dev.command!(motor, "stop")

IOStream(<file /home/pi/ev3/sys/class/tacho-motor/motor0/command>)

In [122]:
Ev3dev.polarity(motor)

"normal"